<a href="https://colab.research.google.com/github/bachaudhry/kaggle-cmi-detect-behavior/blob/main/notebooks/05B-hyper-parameter-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#import os
#os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/Kaggle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Kaggle/CMI - Hyperparameter Tuning/')

In [ ]:
!! pip install catboost optuna wandb

['Collecting catboost',
 '  Downloading catboost-1.2.8-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)',
 'Collecting optuna',
 '  Downloading optuna-4.4.0-py3-none-any.whl.metadata (17 kB)',
 'Requirement already satisfied: wandb in /usr/local/lib/python3.11/dist-packages (0.21.0)',
 'Requirement already satisfied: graphviz in /usr/local/lib/python3.11/dist-packages (from catboost) (0.21)',
 'Requirement already satisfied: matplotlib in /usr/local/lib/python3.11/dist-packages (from catboost) (3.10.0)',
 'Requirement already satisfied: numpy<3.0,>=1.16.0 in /usr/local/lib/python3.11/dist-packages (from catboost) (2.0.2)',
 'Requirement already satisfied: pandas>=0.24 in /usr/local/lib/python3.11/dist-packages (from catboost) (2.2.2)',
 'Requirement already satisfied: scipy in /usr/local/lib/python3.11/dist-packages (from catboost) (1.15.3)',
 'Requirement already satisfied: plotly in /usr/local/lib/python3.11/dist-packages (from catboost) (5.24.1)',
 'Requirement already satisfi

In [ ]:
import pandas as pd
import numpy as np
import os
import catboost as cat
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import warnings
from functools import partial

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

# --- Pathing and Experiment Tracking Setup ---
from src.tracking import ExperimentTracker
from src.config import USE_WANDB, WANDB_PROJECT, WANDB_ENTITY

tracker = ExperimentTracker(
    use_wandb=USE_WANDB,
    wandb_project_name=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY
)

SEED = 42


In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 500, "display.max_columns", 1000):
        display(df)

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Kaggle/CMI - Hyperparameter Tuning/data'

In [ ]:
# Load the raw data and create helper maps
train_sensor = pd.read_csv(os.path.join(os.path.expanduser(DATA_PATH), 'train.csv'))
train_demos = pd.read_csv(os.path.join(os.path.expanduser(DATA_PATH), 'train_demographics.csv'))
train_df = pd.merge(train_sensor, train_demos, on='subject', how='left')
metadata = train_df[['gesture', 'sequence_type']].drop_duplicates()
gesture_to_seq_type_map = metadata.set_index('gesture')['sequence_type'].to_dict()
gesture_map = {label: i for i, label in enumerate(metadata['gesture'].unique())}
inv_gesture_map = {i: label for label, i in gesture_map.items()}

In [ ]:
# Load features feather file and permutation importance csv
features_w4_df = pd.read_feather(DATA_PATH + "/features_4n.fea")


features_w4_df.head()

,sequence_id,subject,gesture,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm,acc_x_mean_Gesture,acc_x_mean_Transition,acc_x_std_Gesture,acc_x_std_Transition,acc_x_min_Gesture,acc_x_min_Transition,acc_x_max_Gesture,acc_x_max_Transition,acc_x_skew_Gesture,acc_x_skew_Transition,acc_y_mean_Gesture,acc_y_mean_Transition,acc_y_std_Gesture,acc_y_std_Transition,acc_y_min_Gesture,acc_y_min_Transition,acc_y_max_Gesture,acc_y_max_Transition,acc_y_skew_Gesture,acc_y_skew_Transition,acc_z_mean_Gesture,acc_z_mean_Transition,acc_z_std_Gesture,acc_z_std_Transition,acc_z_min_Gesture,acc_z_min_Transition,acc_z_max_Gesture,acc_z_max_Transition,acc_z_skew_Gesture,acc_z_skew_Transition,rot_w_mean_Gesture,rot_w_mean_Transition,rot_w_std_Gesture,rot_w_std_Transition,rot_w_min_Gesture,rot_w_min_Transition,rot_w_max_Gesture,rot_w_max_Transition,rot_w_skew_Gesture,rot_w_skew_Transition,rot_x_mean_Gesture,rot_x_mean_Transition,rot_x_std_Gesture,rot_x_std_Transition,rot_x_min_Gesture,rot_x_min_Transition,rot_x_max_Gesture,rot_x_max_Transition,rot_x_skew_Gesture,rot_x_skew_Transition,rot_y_mean_Gesture,rot_y_mean_Transition,rot_y_std_Gesture,rot_y_std_Transition,rot_y_min_Gesture,rot_y_min_Transition,rot_y_max_Gesture,rot_y_max_Transition,rot_y_skew_Gesture,rot_y_skew_Transition,rot_z_mean_Gesture,rot_z_mean_Transition,rot_z_std_Gesture,rot_z_std_Transition,rot_z_min_Gesture,rot_z_min_Transition,rot_z_max_Gesture,rot_z_max_Transition,rot_z_skew_Gesture,rot_z_skew_Transition,thm_1_mean_Gesture,thm_1_mean_Transition,thm_1_std_Gesture,thm_1_std_Transition,thm_1_min_Gesture,thm_1_min_Transition,thm_1_max_Gesture,thm_1_max_Transition,thm_1_skew_Gesture,thm_1_skew_Transition,thm_2_mean_Gesture,thm_2_mean_Transition,thm_2_std_Gesture,thm_2_std_Transition,thm_2_min_Gesture,thm_2_min_Transition,thm_2_max_Gesture,thm_2_max_Transition,thm_2_skew_Gesture,thm_2_skew_Transition,thm_3_mean_Gesture,thm_3_mean_Transition,thm_3_std_Gesture,thm_3_std_Transition,thm_3_min_Gesture,thm_3_min_Transition,thm_3_max_Gesture,thm_3_max_Transition,thm_3_skew_Gesture,thm_3_skew_Transition,thm_4_mean_Gesture,thm_4_mean_Transition,thm_4_std_Gesture,thm_4_std_Transition,thm_4_min_Gesture,thm_4_min_Transition,thm_4_max_Gesture,thm_4_max_Transition,thm_4_skew_Gesture,thm_4_skew_Transition,thm_5_mean_Gesture,thm_5_mean_Transition,thm_5_std_Gesture,thm_5_std_Transition,thm_5_min_Gesture,thm_5_min_Transition,thm_5_max_Gesture,thm_5_max_Transition,thm_5_skew_Gesture,thm_5_skew_Transition,acc_mag_mean_Gesture,acc_mag_mean_Transition,acc_mag_std_Gesture,acc_mag_std_Transition,acc_mag_min_Gesture,acc_mag_min_Transition,acc_mag_max_Gesture,acc_mag_max_Transition,acc_mag_skew_Gesture,acc_mag_skew_Transition,rot_mag_mean_Gesture,rot_mag_mean_Transition,rot_mag_std_Gesture,rot_mag_std_Transition,rot_mag_min_Gesture,rot_mag_min_Transition,rot_mag_max_Gesture,rot_mag_max_Transition,rot_mag_skew_Gesture,rot_mag_skew_Transition,jerk_mean_Gesture,jerk_mean_Transition,jerk_std_Gesture,jerk_std_Transition,jerk_min_Gesture,jerk_min_Transition,jerk_max_Gesture,jerk_max_Transition,jerk_skew_Gesture,jerk_skew_Transition,thm_grad_1_2_mean_Gesture,thm_grad_1_2_mean_Transition,thm_grad_1_2_std_Gesture,thm_grad_1_2_std_Transition,thm_grad_1_2_min_Gesture,thm_grad_1_2_min_Transition,thm_grad_1_2_max_Gesture,thm_grad_1_2_max_Transition,thm_grad_1_2_skew_Gesture,thm_grad_1_2_skew_Transition,thm_grad_2_3_mean_Gesture,thm_grad_2_3_mean_Transition,thm_grad_2_3_std_Gesture,thm_grad_2_3_std_Transition,thm_grad_2_3_min_Gesture,thm_grad_2_3_min_Transition,thm_grad_2_3_max_Gesture,thm_grad_2_3_max_Transition,thm_grad_2_3_skew_Gesture,thm_grad_2_3_skew_Transition,thm_grad_3_4_mean_Gesture,thm_grad_3_4_mean_Transition,thm_grad_3_4_std_Gesture,thm_grad_3_4_std_Transition,thm_grad_3_4_min_Gesture,thm_grad_3_4_min_Transition,thm_grad_3_4_max_Gesture,thm_grad_3_4_max_Transition,thm_grad_3_4_skew_Gesture,thm_grad_3_4_skew_Transition,thm_grad_4_5_mean_Gesture,thm_grad_4_5_mean_Transition,thm_grad_4_5_s

In [ ]:
def average_f1_score(y_true_encoded, y_pred_proba):
    """
    Calculates the official competition F1 score.

    Args:
        y_true_encoded: True labels, integer encoded.
        y_pred_proba: Predicted probabilities from the model.
    """
    # Get predicted labels by finding the class with the highest probability
    y_pred_encoded = np.argmax(y_pred_proba, axis=1)

    # Map integer-encoded labels back to the string representations
    y_true_str = pd.Series(y_true_encoded).map(inv_gesture_map)
    y_pred_str = pd.Series(y_pred_encoded).map(inv_gesture_map)

    # Binary F1
    y_true_binary = y_true_str.map(gesture_to_seq_type_map)
    y_pred_binary = y_true_str.map(gesture_to_seq_type_map)
    binary_f1 = f1_score(y_true_binary, y_pred_binary, pos_label='Target', average='binary')

    # Macro F1 (collaped non-target class)
    def collapse_non_target(gesture):
        return 'non_target' if gesture_to_seq_type_map[gesture] == 'Non-Target' else gesture

    y_true_collapsed = y_true_str.apply(collapse_non_target)
    y_pred_collapsed = y_pred_str.apply(collapse_non_target)
    macro_f1 = f1_score(y_true_collapsed, y_pred_collapsed, average='macro')

    # Final score = average of the two components
    return (binary_f1 + macro_f1) / 2

Action Plan to Accelerate Tuning

1. Switch to a Fixed Hold-Out Validation Set:

    The Problem: Full CV per trial is too slow.

    The Solution: Instead of re-splitting the data 5 times for every trial, we will perform one single split at the very beginning. We will create a large training set (~80% of the data) and a single, fixed hold-out validation set (~20%).

    The Process: Inside each Optuna trial, the model will train once on the large training set and evaluate once on the hold-out set. This immediately gives us a 5x speedup.

2. Re-introduce GPU Acceleration:

    The Problem: Training on the CPU is still time-consuming for each model fit.

    The Solution: This is the perfect time to leverage your RTX 4080. CatBoost has excellent GPU support. By simply adding task_type: 'GPU' to the model parameters, we can expect another significant speedup on top of the hold-out strategy.

Combined Impact: The 5x speedup from the hold-out set, combined with a potential 2-5x speedup from the GPU, will bring the total tuning time from ~83 hours down to a much more manageable 2-8 hours, well within your 12-hour target.




In [ ]:
# --- Create a Single, Fixed Hold-Out Validation Set ---

print("Splitting data into a single train/validation set for rapid tuning...")

# Prepare data
X = features_w4_df.drop(columns=['sequence_id', 'subject', 'gesture', 'gesture_encoded'])
y = features_w4_df['gesture_encoded']
groups = features_w4_df['subject']

# Use one fold from our trusted CV strategy as the hold-out set
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
train_idx, val_idx = next(cv.split(X, y, groups))

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

print(f"Train set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

Splitting data into a single train/validation set for rapid tuning...
Train set shape: (6623, 343)
Validation set shape: (1528, 343)


In [ ]:
# Define the Optuna objective function - single trial

def objective(trial, X_train, y_train, X_val, y_val):
    """
    This accelerated function trains only ONE model per trial on the GPU
    using a fixed hold-out set.
    """
    params = {
        'iterations': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08, log=True),
        'depth': trial.suggest_int('depth', 5, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2.0, 15.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1.0, 10.0, log=True),
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'random_seed': SEED,
        'verbose': 0,
        # --- GPU ACCELERATION ENABLED ---
        'task_type': 'GPU',
        'devices': '0'
    }

    model = cat.CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=150, verbose=0)

    val_preds_proba = model.predict_proba(X_val)

    # Return the single F1 score for this trial
    return average_f1_score(y_val, val_preds_proba)

In [ ]:
# Create the study object with direction set to 'maximize' for our F1 score
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=SEED)
    # Removing pruner for run 1
)

# Create the objective function with our fixed data
objective_with_data = lambda trial: objective(trial, X_train, y_train, X_val, y_val)

# Start the optimization. 50-100 trials is now feasible.
study.optimize(objective_with_data, n_trials=75, show_progress_bar=True)

# --- Print and Log the Best Results ---
print("\n" + "="*50)
print("Optuna Hyperparameter Tuning Complete")
print(f"Number of finished trials: {len(study.trials)}")

best_trial = study.best_trial
print(f"Best trial's Hold-Out F1 Score: {best_trial.value:.5f}")
print("Best trial's hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

tracker.log_experiment(
    experiment_name="CatBoost-Tuning-Run",
    model_name="CatBoost (Tuned)",
    feature_wave="Wave 4 (Fixed)",
    cv_score=best_trial.value,
    params=best_trial.params,
    notes="Best parameters found from a 75-trial Optuna study using a fixed hold-out set and GPU acceleration."
)

[I 2025-07-15 09:30:31,145] A new study created in memory with name: no-name-3224fc9b-c417-4bef-8508-13eea6fa4539


  0%|          | 0/75 [00:00<?, ?it/s]

[I 2025-07-15 09:33:10,927] Trial 0 finished with value: 0.8399394212567655 and parameters: {'learning_rate': 0.021789307659775742, 'depth': 8, 'l2_leaf_reg': 8.741169448864675, 'bagging_temperature': 0.5986584841970366, 'random_strength': 1.4322493718230251}. Best is trial 0 with value: 0.8399394212567655.
[I 2025-07-15 09:33:37,930] Trial 1 finished with value: 0.8213458832583289 and parameters: {'learning_rate': 0.013831748681407519, 'depth': 5, 'l2_leaf_reg': 11.454791487656054, 'bagging_temperature': 0.6011150117432088, 'random_strength': 5.105903209394756}. Best is trial 0 with value: 0.8399394212567655.
[I 2025-07-15 09:36:18,933] Trial 2 finished with value: 0.8245469999485663 and parameters: {'learning_rate': 0.010437335666720523, 'depth': 8, 'l2_leaf_reg': 10.702082748695268, 'bagging_temperature': 0.21233911067827616, 'random_strength': 1.5199348301309807}. Best is trial 0 with value: 0.8399394212567655.
[I 2025-07-15 09:37:08,379] Trial 3 finished with value: 0.832993747526

OSError: Cannot save file into a non-existent directory: '/root/code/kaggle/kaggle-cmi-detect-behavior'

In [ ]:
# --- Define the Robust Objective Function for Optuna ---

def objective_cv(trial, X, y, groups, scorer):
    """
    This robust function runs a full 5-fold cross-validation for each Optuna trial.
    It uses GPU acceleration to make this process feasible.
    """
    # Define the hyperparameter search space
    params = {
        'iterations': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08, log=True),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2.0, 20.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1.0, 10.0, log=True),
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'random_seed': SEED,
        'verbose': 0,
        'task_type': 'GPU', # Enable GPU acceleration
        'devices': '0'
    }

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = cat.CatBoostClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=150)

        val_preds_proba = model.predict_proba(X_val)
        fold_score = scorer(y_val, val_preds_proba)
        fold_scores.append(fold_score)

        # Report intermediate results to the pruner
        trial.report(fold_score, fold)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return np.mean(fold_scores)

In [ ]:
# --- Prepare data for tuning ---
X = features_w4_df.drop(columns=['sequence_id', 'subject', 'gesture', 'gesture_encoded'])
y = features_w4_df['gesture_encoded']
groups = features_w4_df['subject']

In [ ]:
SEED = 42

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=2) # Prune after 2 full folds
)

# Create the self-contained scorer once
scorer_func = partial(
    average_f1_score,
    inv_gesture_map=inv_gesture_map,
    gesture_to_seq_type_map=gesture_to_seq_type_map
)

scorer_func = lambda y_true, y_pred: average_f1_score(y_true, y_pred)
final_objective = lambda trial: objective_cv(trial, X, y, groups, scorer_func)

study.optimize(final_objective, n_trials=50, show_progress_bar=True)

# --- Print and Log the Best Results ---
print("\n" + "="*50)
print("Final Robust Optuna Tuning Complete")
print(f"Number of finished trials: {len(study.trials)}")

best_trial = study.best_trial
print(f"Best trial's Mean CV F1 Score: {best_trial.value:.5f}")
print("Best trial's hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

tracker.log_experiment(
    experiment_name="CatBoost-Robust-Tuned-Final",
    model_name="CatBoost (Robust Tuned)",
    feature_wave="Wave 4 (Fixed)",
    cv_score=best_trial.value,
    params=best_trial.params,
    notes="Best parameters found from a 75-trial Optuna study using 5-fold CV per trial."
)

[I 2025-07-15 12:07:34,918] A new study created in memory with name: no-name-4242e2a5-f9b8-43fe-ab83-dbc6f198a836


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-07-15 12:20:56,063] Trial 0 finished with value: 0.7998662419089287 and parameters: {'learning_rate': 0.021789307659775742, 'depth': 8, 'l2_leaf_reg': 10.790061933340453, 'bagging_temperature': 0.5986584841970366, 'random_strength': 1.4322493718230251}. Best is trial 0 with value: 0.7998662419089287.
[I 2025-07-15 12:22:20,774] Trial 1 finished with value: 0.7844952068876252 and parameters: {'learning_rate': 0.013831748681407519, 'depth': 4, 'l2_leaf_reg': 14.696236810540892, 'bagging_temperature': 0.6011150117432088, 'random_strength': 5.105903209394756}. Best is trial 0 with value: 0.7998662419089287.
[I 2025-07-15 12:35:48,649] Trial 2 finished with value: 0.789955218884516 and parameters: {'learning_rate': 0.010437335666720523, 'depth': 8, 'l2_leaf_reg': 13.597924843182254, 'bagging_temperature': 0.21233911067827616, 'random_strength': 1.5199348301309807}. Best is trial 0 with value: 0.7998662419089287.
[I 2025-07-15 12:38:10,216] Trial 3 finished with value: 0.796260216531

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: b-a-chaudhry (b-a-chaudhry-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


cv_score,▁
cv_score,0.81155
feature_wave,Wave 4 (Fixed)
model_name,CatBoost (Robust Tun...


Experiment 'CatBoost-Robust-Tuned-Final' logged to W&B
